In [1]:
import pandas as pd
import json
import os

# Function to process only Test match data from the JSON and generate DataFrame
def create_test_dataframe_from_json():
    folder_path = "tests_json"
    test_match_data = []

    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith(".json"):
                file_path = os.path.join(folder_path, filename)
                try:
                    with open(file_path, "r") as file:
                        data = json.load(file)
                        if isinstance(data, dict):
                            match_info = data.get('info', {})
                            teams = match_info.get("teams", [])
                            match_type = match_info.get("match_type", "Unknown")
                            match_date = match_info.get('dates', ["Unknown"])[0]
                            outcome = match_info.get('outcome')
                            winner = match_info.get('outcome', {}).get('winner', 'Unknown')
                            player_of_match = ", ".join(match_info.get("player_of_match", []))
                            
                            # Extract player statistics
                            player_stats = {}
                            for team in teams:
                                players = match_info.get("players", {}).get(team, [])
                                for player in players:
                                    player_stats[player] = {"runs": 0, "wickets": 0}
                            
                            innings_data = []
                            for innings in data.get("innings", []):
                                team = innings.get("team", "Unknown")
                                for over in innings.get("overs", []):
                                    over_number = over.get("over", "Unknown")
                                    for delivery in over.get("deliveries", []):
                                        batter = delivery.get("batter", "Unknown")
                                        bowler = delivery.get("bowler", "Unknown")
                                        runs_batter = delivery["runs"].get("batter", 0)
                                        extras_noballs = delivery["extras"].get("noballs", 0) if "extras" in delivery else 0
                                        runs_total = runs_batter + extras_noballs
                                        
                                        if batter in player_stats:
                                            player_stats[batter]["runs"] += runs_batter
                                        if bowler in player_stats:
                                            player_stats[bowler]["wickets"] += 1

                                        # Add row for each delivery
                                        test_match_data.append({
                                            "match_date": match_date,
                                            "match_type": match_type,
                                            "teams": ", ".join(teams),
                                            "match_result": winner,
                                            "outcome":outcome,
                                            "player_of_match": player_of_match,
                                            "player": batter,
                                            "runs_batter": runs_batter,
                                            "wickets_bowler": player_stats[bowler]["wickets"],
                                            "extras_noballs": extras_noballs,
                                            "runs_total": runs_total,
                                            "over": over_number,
                                            "team": team
                                        })
                            
                            # Add player statistics to match data
                            for player, stats in player_stats.items():
                                test_match_data.append({
                                    "match_date": match_date,
                                    "match_type": match_type,
                                    "teams": ", ".join(teams),
                                    "match_result": winner,
                                    "outcome":outcome,
                                    "player_of_match": player_of_match,
                                    "player": player,
                                    "runs_batter": stats["runs"],
                                    "wickets_bowler": stats["wickets"],
                                    "extras_noballs": 0,
                                    "runs_total": stats["runs"],
                                    "over": "N/A",
                                    "team": "All"
                                })
                        else:
                            print(f"Skipping invalid data format in file {filename}")
                except json.JSONDecodeError:
                    print(f"Error decoding JSON from file {filename}")
                except Exception as e:
                    print(f"Error processing file {filename}: {e}")
                print(f"Processed matches from {file_path}")
    else:
        print(f"Folder {folder_path} does not exist!")

    # Create DataFrame
    if test_match_data:
        test_matches_df = pd.DataFrame(test_match_data)
        test_matches_df.to_csv("test_matches.csv", index=False)
        print("Test DataFrame created and saved as test_matches.csv.")
    else:
        print("No valid Test match data found!")

# Example usage
if __name__ == "__main__":
    create_test_dataframe_from_json()

Processed matches from tests_json\1000851.json
Processed matches from tests_json\1000853.json
Processed matches from tests_json\1000855.json
Processed matches from tests_json\1000881.json
Processed matches from tests_json\1000883.json
Processed matches from tests_json\1000885.json
Processed matches from tests_json\1019985.json
Processed matches from tests_json\1019987.json
Processed matches from tests_json\1019993.json
Processed matches from tests_json\1019995.json
Processed matches from tests_json\1020041.json
Processed matches from tests_json\1020043.json
Processed matches from tests_json\1020045.json
Processed matches from tests_json\1022593.json
Processed matches from tests_json\1022595.json
Processed matches from tests_json\1022597.json
Processed matches from tests_json\1022599.json
Processed matches from tests_json\1024041.json
Processed matches from tests_json\1024043.json
Processed matches from tests_json\1029825.json
Processed matches from tests_json\1029827.json
Processed mat

In [8]:
import streamlit as st
import matplotlib.pyplot as plt
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('cricket_matches.db')
cursor = conn.cursor()

# Function to fetch data for visualizations
def fetch_data(query):
    return pd.read_sql(query, conn)

# Streamlit setup
st.title("Cricket Data Insights")
st.sidebar.title("Select Insight")
def extract_margin(match_result):
    if match_result and "by" in match_result:
        margin_str = match_result.split("by")[1].strip()
        # Extract the number from the string (could be either 'runs' or 'wickets')
        margin = ''.join([char for char in margin_str if char.isdigit()])
        return int(margin) if margin else None
    return None

# 1. Top 10 Batsmen by Total Runs in ODI Matches
if st.sidebar.checkbox('Top 10 Batsmen by Total Runs in ODI Matches'):
    query = """
    SELECT player_of_match,
           IFNULL(SUM(runs_batter), 0) AS total_runs
    FROM odi_matches
    WHERE player_of_match != 'Unknown'
    GROUP BY player_of_match
    ORDER BY total_runs DESC LIMIT 10;
    """
    data = fetch_data(query)

    st.subheader('Top 10 Batsmen by Total Runs in ODI Matches')
    st.dataframe(data)

    # Creating a figure and axis before plotting
    fig, ax = plt.subplots(figsize=(10,6))
    ax.barh(data['player_of_match'], data['total_runs'], color='skyblue')
    ax.set_xlabel('Total Runs')
    ax.set_title('Top 10 Batsmen by Total Runs in ODI Matches')
    st.pyplot(fig)

# 2. Leading Wicket-Takers in T20 Matches
if st.sidebar.checkbox('Leading Wicket-Takers in T20 Matches'):
    query = """
    SELECT player_of_match,
           IFNULL(SUM(wickets_bowler), 0) AS total_wickets
    FROM t20_matches
    WHERE player_of_match != 'Unknown'
    GROUP BY player_of_match
    ORDER BY total_wickets DESC LIMIT 10;
    """
    data = fetch_data(query)

    st.subheader('Leading Wicket-Takers in T20 Matches')
    st.dataframe(data)

    # Creating a figure and axis before plotting
    fig, ax = plt.subplots(figsize=(10,6))
    ax.barh(data['player_of_match'], data['total_wickets'], color='salmon')
    ax.set_xlabel('Total Wickets')
    ax.set_title('Leading Wicket-Takers in T20 Matches')
    st.pyplot(fig)

# 3. Total Number of Centuries Across All Match Types
if st.sidebar.checkbox('Total Number of Centuries Across All Match Types'):
    query = """
    SELECT COUNT(*) AS total_centuries
    FROM (
        SELECT runs_batter FROM test_matches WHERE runs_batter >= 100
        UNION ALL
        SELECT runs_batter FROM odi_matches WHERE runs_batter >= 100
        UNION ALL
        SELECT runs_batter FROM t20_matches WHERE runs_batter >= 100
        UNION ALL
        SELECT runs_batter FROM it20_matches WHERE runs_batter >= 100
    );
    """
    data = fetch_data(query)

    st.subheader('Total Number of Centuries Across All Match Types')
    st.write(f'Total Centuries: {data["total_centuries"][0]}')


# 4. Top 10 Batsmen by Total Runs in Test Matches
if st.sidebar.checkbox('Top 10 Batsmen by Total Runs in Test Matches'):
    query = """
    SELECT player_of_match,
           IFNULL(SUM(runs_batter), 0) AS total_runs
    FROM test_matches
    WHERE player_of_match != 'Unknown'
    GROUP BY player_of_match
    ORDER BY total_runs DESC LIMIT 10;
    """
    data = fetch_data(query)

    st.subheader('Top 10 Batsmen by Total Runs in Test Matches')
    st.dataframe(data)

    # Creating a figure and axis before plotting
    fig, ax = plt.subplots(figsize=(10,6))
    ax.barh(data['player_of_match'], data['total_runs'], color='lightcoral')
    ax.set_xlabel('Total Runs')
    ax.set_title('Top 10 Batsmen by Total Runs in Test Matches')
    st.pyplot(fig)

# 5. Best Bowling Performance by Player in T20 Matches
if st.sidebar.checkbox('Best Bowling Performance by Player in T20 Matches'):
    query = """
    SELECT player_of_match,
           MAX(wickets_bowler) AS best_bowling_performance
    FROM t20_matches
    WHERE player_of_match != 'Unknown'
    GROUP BY player_of_match
    ORDER BY best_bowling_performance DESC LIMIT 10;
    """
    data = fetch_data(query)

    st.subheader('Best Bowling Performance by Player in T20 Matches')
    st.dataframe(data)

    # Creating a figure and axis before plotting
    fig, ax = plt.subplots(figsize=(10,6))
    ax.barh(data['player_of_match'], data['best_bowling_performance'], color='darkblue')
    ax.set_xlabel('Best Bowling Performance (Wickets)')
    ax.set_title('Best Bowling Performance by Player in T20 Matches')
    st.pyplot(fig)

# 6. Most Runs in a Single ODI Match (By Player)
if st.sidebar.checkbox('Most Runs in a Single ODI Match (By Player)'):
    query = """
    SELECT player_of_match,
           MAX(runs_batter) AS most_runs
    FROM odi_matches
    WHERE player_of_match != 'Unknown'
    GROUP BY player_of_match
    ORDER BY most_runs DESC LIMIT 10;
    """
    data = fetch_data(query)

    st.subheader('Most Runs in a Single ODI Match (By Player)')
    st.dataframe(data)

# 7. Total Number of Matches Played by Team in ODI Matches
if st.sidebar.checkbox('Total Number of Matches Played by Team in ODI Matches'):
    query = """
    SELECT teams, COUNT(*) AS total_matches
    FROM odi_matches
    WHERE teams != 'Unknown'
    GROUP BY teams
    ORDER BY total_matches DESC LIMIT 10;
    """
    data = fetch_data(query)

    st.subheader('Total Number of Matches Played by Team in ODI Matches')
    st.dataframe(data)

    # Creating a figure and axis before plotting
    fig, ax = plt.subplots(figsize=(10,6))
    ax.barh(data['teams'], data['total_matches'], color='seagreen')
    ax.set_xlabel('Total Matches')
    ax.set_title('Total Number of Matches Played by Team in ODI Matches')
    st.pyplot(fig)

